In [ ]:
import findspark
findspark.init() 

In [ ]:
# Pyspark context creation
from pyspark import SparkContext
sc = SparkContext()
from pyspark import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
# Importing required libraries
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StringType, DoubleType,DateType

import pandas as pd
import plotly.express as px

# Data Processing

In [ ]:
"""['ap-northeast-1.csv','ap-northeast-2.csv','ap-south-1.csv','ap-southeast-1.csv','ap-southeast-2.csv',
 'ca-central-1.csv','eu-central-1.csv','eu-west-1.csv','sa-east-1.csv','us-east-1.csv','us-west-1.csv']"""

file_names = ['ap-northeast-1.csv','ap-northeast-2.csv']

In [ ]:
# Using sqlcontext to load data
data = sqlContext.read.format('csv').options(inferSchema = True).load("data/{}".format(file_names[0]))

# Renaming Column names
data = data.select(date_format(col("_c0"), "dd-MM-yyyy hh:mm:ss").alias("Date"),col("_c1").alias("Instance Type"),
                  col("_c2").alias("OS"),col("_c3").alias("Region"),col("_c4").alias("Price"))

In [ ]:
for files in file_names[1:]:
    print(files)
    temp_data = sqlContext.read.format('csv').options(inferSchema = True).load("data/{}".format(files))
    temp_data = temp_data.select(date_format(col("_c0"), "dd-MM-yyyy hh:mm:ss").alias("Date"),col("_c1").alias("Instance Type"),
                  col("_c2").alias("OS"),col("_c3").alias("Region"),col("_c4").alias("Price"))
    data = data.union(temp_data)

In [ ]:
# dig at data
data.show(5)

In [ ]:
#About data
print("data : ap-northeast-1")
print('Columns --> ',data.columns)
print('Shape --> ',data.count(),",",len(data.columns))

In [ ]:
# Data Description
data.describe().show()

# Data visualisation

In [ ]:
# grouping data by os type and Region and finding average value of Price
df = data.select("Region","OS","Price").groupby("OS","Region").avg("Price").toPandas()

In [ ]:
# Plotting bar plot
px.bar(df,x = "OS",y = "avg(Price)", color = "Region",title = "avg of Price of various OS")

In [ ]:
# grouping data by os type and instance type and finding average value of Price
df = data.select("Instance Type","OS","Price").groupby("OS","Instance Type").avg("Price").toPandas()


In [ ]:
# Plotting bar plot
px.bar(df,x = "OS",y = "avg(Price)", color = "Instance Type",title = "data : northeast1, avg of Price of various OS")

In [ ]:
# Converting to Panda for better visualization
data = data.toPandas()

In [ ]:
# Getting value counts of various instances
idx = data["Instance Type"].value_counts().index
values = data["Instance Type"].value_counts().values
px.bar(x=idx,y=values,color=values,title="Instance Types")

In [ ]:
df = data[data["Instance Type"] == "m3.xlarge"]
px.scatter(df,x="Date",y="Price",color="OS",title="data : northeast1, instance :m3.xlarge ")

In [ ]:
data_win = df[df["OS"]=="Windows"]
px.line(data_win,x="Date",y="Price",title="Price of instance :m3.xlarge, OS: Windows ")

In [ ]:
data_lin = df[df["OS"]=="Linux/UNIX"]
px.line(data_lin,x="Date",y="Price",title="Price of instance :m3.xlarge, OS: Linux/UNIX ")

In [ ]:
data_sus = df[df["OS"]=="SUSE Linux"]
px.line(data_sus,x="Date",y="Price",title="Price of instance :m3.xlarge, OS: SUSE Linux ")

In [ ]:
data.Date = pd.to_datetime(data.Date)
# getting hour of day
data["hour"] = data.Date.dt.hour
# calculating mean price of every hour
pricedist = data.groupby(data.hour).mean()

In [ ]:
# Plotting mean price per hour
px.line(pricedist, x = pricedist.index.values,y ="Price",title="Price variations/hour")

In [ ]:
# getting month
data["month"] = data.Date.dt.month
# calculating mean price of every month
pricedist = data.groupby(data.month).mean()

In [ ]:
# Plotting mean price per month
px.line(pricedist, x = pricedist.index.values,y ="Price",title="Price variations/month")

In [ ]:
# Saving data to csv format
data.to_csv("finalData.csv",index=None)